In [1]:
import os

import matplotlib.pyplot as plt
import tifffile as tf
import pandas as pd
import numpy as np
import PIL

import skimage.measure as measure

import line_utils

In [2]:
# Establish the path the Excel file
base_path = "/Users/zachcm/Documents/Projects/ExM_Nadja/01_Macros_Analysis/"
workbook = "20241008_septin2_tubulin.xlsx"
workbook_path = os.path.join(base_path, workbook)
expected_colnames=['Distance_(microns)', 'MTs', 'septin2', 'DAPI']

In [3]:
# Load the table of contents
toc = pd.read_excel(workbook_path, sheet_name="ToC + P-t-p", header=2)

In [4]:
# load FWHM_along, which contains other statistics
metrics = pd.read_excel(workbook_path, sheet_name="FWHM_along", header=0).dropna(axis=0)

In [5]:
metrics = line_utils.merge_df_information(toc, 
                                          metrics, 
                                          id_key="Label", 
                                          mapped_keys=["dX (µm)", "dX (pxl)", "X1", "X2", "Angle","X","Y","length"])
metrics = metrics[~metrics['X1'].isna()]
# metrics = metrics.dropna()

In [6]:
# Now let's get the original images...
import glob

max_proj_path = "/Volumes/Ries_Ewers/Septin2-GFP/MaxIPs"
max_proj_files = glob.glob(max_proj_path+"/*.tif")

# ...and associate the file with each metrics entry
metrics["filename"] = ""
for i, ml in metrics.iterrows():
    for fn in max_proj_files:
        if ml["Label"] in fn:
            metrics.loc[i, "filename"] = fn
            break

In [7]:
groups = metrics.groupby('Unnamed: 0')

plot_stack = None
n_groups = len(groups)
n_ch = 3
for group, tup in enumerate(groups):
    name, entries = tup
    n_group = len(entries)
    for i, ml in entries.iterrows():
        # Get the image associated with this row
        im = tf.imread(ml["filename"])

        # get x, y, angle, length for this row
        x, y, angle, length = ml[["X", "Y", "Angle", "length"]]

        # Compute the line end points
        xl, xu, yl, yu = line_utils.get_line_profile_endpoints(x, y, angle, length)

        # Extract the line profiles
        chs = measure.profile_line(im.T, 
                                [xl, yu], 
                                [xu, yl], 
                                linewidth=25)

        mt, septin, dapi = chs.T

        # Get the rescaling to the septin endpoints
        xx_norm = line_utils.rescale_inds(len(mt), int(ml["X1"]), int(ml["X2"]))
        xx_norm = xx_norm - 0.5

        for ch, profile in enumerate([mt, septin, dapi]):
            profile_norm = profile/profile.sum()
            fig, ax = plt.subplots(1,1, figsize=(5,5))

            # Set limits
            ax.set_xlim([-5,5])
            ax.set_ylim([0,0.01])

            # Move left y-axis and bottom x-axis to centre, passing through (0,0)
            ax.spines['left'].set_position('center')
            # ax.spines['bottom'].set_position('center')

            # Eliminate upper and right axes
            ax.spines['right'].set_color('none')
            ax.spines['top'].set_color('none')

            # Show ticks in the left and lower axes only
            ax.xaxis.set_ticks_position('bottom')
            ax.yaxis.set_ticks_position('left')

            # plot
            ax.plot(xx_norm, profile_norm, c='k')

            fig.tight_layout()

            plt.savefig("foo.png")

            plt.close()

            plot_im = np.array(PIL.Image.open("foo.png").convert('L'))
            
            if plot_stack is None:
                # TCYX
                # Do this the first time b/c we don't know plot_im size
                plot_stack = np.zeros((n_groups, n_ch, plot_im.shape[0], plot_im.shape[1]), dtype=plot_im.dtype)
            
            plot_stack[group, ch, ...] += ((1/n_group)*plot_im).astype(np.uint8)

In [8]:
tf.imwrite('pseudotime_profiles.ome.tif', plot_stack, metadata={'axes': 'TCYX'}, dtype=plot_stack.dtype)

In [9]:
for group, tup in enumerate(groups):
    name, entries = tup
    n_group = len(entries)
    print(name, n_group)

CS 16
RC 1
RS 24
SM 34
